In [88]:
from enum import Enum, auto
import snap
import os.path
import itertools
import numpy as np

import util

# Global Variables

In [89]:
__no_label__ = "NO_LABEL"

In [90]:
# Constants used in the paper:
# NODE_WEIGHT_EVALUATION("NUMBER_OF_NODES_INSIDE_OF_SN"),
# NODE_WEIGHT_FREQUENCY("NUMBER_OF_SN_INSIDE_OF_HN"),
# NODE_WEIGHT_EVALUATION_AVG("AVG_WEIGHT_ON_SN"),
# PERCENTAGE("PERCENTAGE"),
# NUMBER_OF_INNER_EDGES("NUMBER_OF_INNER_EDGES"),
# LABEL("LABEL"),
# EDGE_RATIO("EDGE_RATIO"),
# GROUPING("GROUPING"),
# REACHABILITY_COUNT("REACH_NUMBER_OF_INNER_PATHS"),
# PATH_OUT("REACH_PATH_OUT_BY_LABEL"),                      # Not used in og repo
# PATH_IN("REACH_PATH_IN_BY_LABEL"),                        # Not used in og repo
# EDGE_WEIGHT("EDGE_WEIGHT"),
# PARTICIPATION_LABEL("PARTICIPATION_LABEL"),
# TRAVERSAL_FRONTIERS("TRAVERSAL_FRONTIERS");

class AttributeConstants(str, Enum):
    # General attributes for all graphs 
    EDGE_LABEL = "EDGE_LABEL" # indicates the edge label, e.g. KNOWS, LIKES...

    # Attributes for the OG graph
    SUPER_NODE_ID = "SUPER_NODE_ID" # indicates the super node id that the node belongs to
    
    # Attributes for the Evaluation graph
    HYPER_NODE_ID = "HYPER_NODE_ID" # indicates the hyper node id that the super node belongs to
    SN_NODE_WEIGHT = "NUMBER_OF_INNER_NODES" # indicates the number of inner nodes the super node has NOTE: Maybe change this to no_of_inner_base_nodes
    SN_EDGE_WEIGHT = "NUMBER_OF_INNER_EDGES" # indicates the number of inner edges the super node has NOTE: Maybe change this to no_of_inner_base_edges
    
    # Inner-Connectivity - Each of the following attributes will exist for each
    # l-label, i.e., we would have PERCENTAGE_KNOWS, PERCENTAGE_LIKES, for a 
    # graph with l-labels = {KNOWS, LIKES}.
    SN_LABEL_PERCENTAGE = "LABEL_PERCENTAGE_" # percent of l-labeled inner edges
    SN_LABEL_REACH = "LABEL_REACH_" # number of inner node pairs connected with an l-labeled inner edge

    SE_EDGE_WEIGHT = "SE_NUMBER_OF_EDGES" # indicates the number of inner edges the super edge has
    # SE_EDGE_RATIO CHECK used by the author to calculate participation, but I don't think it's needed

    # Attributes for the Merge graph
    HN_NODE_WEIGHT = "NUMBER_OF_INNER_SUPER_NODES" # indicates the number of inner super nodes the hyper node has
    HN_AVG_NODE_WEIGHT = "AVG_NUMBER_OF_INNER_NODES" # for a given hyper node, this indicates the average number of inner nodes its super nodes have

    HE_EDGE_WEIGHT = "HE_NUMBER_OF_EDGES"
    
# class NodeAttributeConstants(str, Enum):
    # Compression
    

    # NODE_WEIGHT_AVG = "AVG_WEIGHT_ON_SN"
    
    # NODE_WEIGHT_FREQUENCY = "NUMBER_OF_SN_INSIDE_OF_HN"
    
    # LABEL = "LABEL"
    # GROUPING = "GROUPING"

    # EDGE_RATIO = "EDGE_RATIO"
    
    # PARTICIPATION_LABEL = "PARTICIPATION_LABEL"
    # TRAVERSAL_FRONTIERS = "TRAVERSAL_FRONTIERS"

# class EdgeAttributeConstants(str, Enum):


# class LabelAttributeConstants(str, Enum):
    

# Load datasets

In [91]:
class EdgeFileColumns(Enum):
    SRC_COL = auto()
    DST_COL = auto()
    SRC_NODE = auto()
    DST_NODE = auto()
    EDGE = auto()


class GMarkUseCase(Enum):
    shop = auto()
    social_network = auto()
    test = auto()
    uniprot = auto()


# Generic funtion to read edge file into a network
# TODO: 
# allow any graph type not only snap.TNEANet
# check if binary exists, if so, load that (rename function to make graph/net)
def edge_file_to_network(filename, ordered_attributes, tab_separated=False, has_title_line=snap.TBool(False), dump=False):
    context = snap.TTableContext()

    src_node_attr_v = snap.TStrV()
    dst_node_attr_v = snap.TStrV()
    edge_attr_v = snap.TStrV()
    schema = snap.Schema()
    for attr_category, (attr_name, attr_type) in ordered_attributes.items():
        if attr_category is EdgeFileColumns.SRC_COL:
            src_col = attr_name
        elif attr_category is EdgeFileColumns.DST_COL:
            dst_col = attr_name
        elif attr_category is EdgeFileColumns.SRC_NODE:
            src_node_attr_v.Add(attr_name)
        elif attr_category is EdgeFileColumns.DST_NODE:
            dst_node_attr_v.Add(attr_name)
        elif attr_category is EdgeFileColumns.EDGE:
            edge_attr_v.Add(attr_name)
        schema.Add(snap.TStrTAttrPr(attr_name, attr_type))
    
    if tab_separated:
        separator = '\t'
    else:
        separator = ' '

    table = snap.TTable.LoadSS(schema, filename, context, separator, has_title_line)

    # network will be an object of type snap.TNEANet
    network = table.ToNetwork(snap.TNEANet, src_col, dst_col, src_node_attr_v, dst_node_attr_v, edge_attr_v, snap.aaFirst)

    if dump:
        network.Dump()

    # Save to binary
    outfile = filename + ".bin"
    FOut = snap.TFOut(outfile)
    table.Save(FOut)
    FOut.Flush()

    return network


def make_residence_hall_network(dump=False):
    filename = "data/moreno_oz/out.moreno_oz_oz"

    edge_file_column_info = {
        EdgeFileColumns.SRC_COL : ('SRC_COL', snap.atInt),
        EdgeFileColumns.DST_COL : ('DST_COL', snap.atInt),
        EdgeFileColumns.EDGE : (AttributeConstants.EDGE_LABEL.value, snap.atInt)        
    }

    return edge_file_to_network(filename, edge_file_column_info, dump)


def make_pg_paper_network(dump=False):
    filename = "data/example/pg_paper.txt"

    edge_file_column_info = {
        EdgeFileColumns.SRC_COL : ('SRC_COL', snap.atInt),
        EdgeFileColumns.DST_COL : ('DST_COL', snap.atInt),
        EdgeFileColumns.EDGE : (AttributeConstants.EDGE_LABEL.value, snap.atStr)        
    }

    return edge_file_to_network(filename, edge_file_column_info, dump)
    

def make_author_repo_network(gmark_use_case, dump=False):
    if gmark_use_case is GMarkUseCase.shop:
        filename = "data/author_repo/shop_dataset.txt"
    elif gmark_use_case is GMarkUseCase.social_network:
        filename = "data/author_repo/social_network_dataset.txt"
    elif gmark_use_case is GMarkUseCase.test:
        filename = "data/author_repo/test_dataset.txt"
    elif gmark_use_case is GMarkUseCase.uniprot:
        filename = "data/author_repo/uniprot_dataset.txt"

    # dict maintains insertion order
    # add the attributes according to the order they are in their file
    edge_file_column_info = {
        EdgeFileColumns.SRC_COL : ('SRC_COL', snap.atInt),
        EdgeFileColumns.EDGE : (AttributeConstants.EDGE_LABEL.value, snap.atStr),
        EdgeFileColumns.DST_COL : ('DST_COL', snap.atInt)
    }

    return edge_file_to_network(filename, edge_file_column_info, dump)


In [96]:
def print_type_attributes(network, id, attr_name_func, attr_value_func):
    attr_names = snap.TStrV()
    getattr(network, attr_name_func)(id, attr_names)
    for attr_name in attr_names:
        val = getattr(network, attr_value_func)(id, attr_name)
        print("--> {}: {}".format(attr_name, val))

def print_all_edge_attributes(network):
    for EI in network.Edges():
        edge_id = EI.GetId()
        src_node_id = EI.GetSrcNId()
        dst_node_id = EI.GetDstNId()
        print("Edge id: {}; ({}) -> ({})".format(
            edge_id, src_node_id,dst_node_id))
        print_type_attributes(network, edge_id, "IntAttrNameEI", 
            "GetIntAttrDatE")
        print_type_attributes(network, edge_id, "FltAttrNameEI",
            "GetFltAttrDatE")
        print_type_attributes(network, edge_id, "StrAttrNameEI",
            "GetStrAttrDatE")

def print_all_node_attributes(network):
    for NI in network.Nodes():
        node_id = NI.GetId()
        print("Node id: {}".format(node_id))
        print_type_attributes(network, node_id, "IntAttrNameNI", 
            "GetIntAttrDatN")
        print_type_attributes(network, node_id, "FltAttrNameNI",
            "GetFltAttrDatN")
        print_type_attributes(network, node_id, "StrAttrNameNI",
            "GetStrAttrDatN")

def check_merge_graph(network, evaluation_graph, merge_graph):
    se_edge_weight = 0
    for EI in evaluation_graph.Edges():
        se_edge_weight += evaluation_graph.GetFltAttrDatE(
            EI, AttributeConstants.SE_EDGE_WEIGHT.value)

    sn_edge_weight = 0
    for NI in evaluation_graph.Nodes():
        sn_edge_weight += evaluation_graph.GetFltAttrDatN(
            NI, AttributeConstants.SN_EDGE_WEIGHT.value)
    
    he_edge_weight = 0
    for EI in merge_graph.Edges():
        he_edge_weight += merge_graph.GetFltAttrDatE(
            EI, AttributeConstants.HE_EDGE_WEIGHT.value)
    
    # hn_edge_weight = 0
    # for NI in merge_graph.Nodes():
    #     hn_edge_weight += merge_graph.GetFltAttrDatN(
    #         NI, AttributeConstants.HN_EDGE_WEIGHT.value)

    print(f'{network.GetEdges()} -> {se_edge_weight} + {sn_edge_weight} -> {he_edge_weight} + ???')

# Grouping

In [97]:
class Session:
    def __init__(self, network):
        self.network = network
        self.labels_freq = {}   # CHECK: do I use the values or only the keys?
        for EI in network.Edges():
            edge_id = EI.GetId()
            attr_values = snap.TStrV()
            network.AttrValueEI(edge_id, attr_values)
            label = attr_values[0]
            self.labels_freq[label] = self.labels_freq.setdefault(label, 0) + 1
        self.groupings = {}
        self.network.AddIntAttrN(AttributeConstants.SUPER_NODE_ID.value)

    # The 1st attribute of an edge is the edge label
    def __get_edge_ids_per_label(self):
        labels = {}
        for EI in self.network.Edges():
            edge_id = EI.GetId()
            label = self.network.GetStrAttrDatE(
                EI, AttributeConstants.EDGE_LABEL.value)
            labels.setdefault(label, snap.TIntV()).append(edge_id)
        return labels

    def compute_groupings(self):
        labels_edge_ids = self.__get_edge_ids_per_label()
        node_ids_in_groupings = snap.TIntV()
        for label, edge_ids in sorted(labels_edge_ids.items(), 
                                        key=lambda item: len(item[1]), 
                                        reverse=True):
            grouping = self.network.ConvertESubGraph(snap.TNEANet, edge_ids)
            # We cannot use DelNodes; an exception is thrown if we try to remove a 
            # node that is not there. There is not way to continue after the 
            # exception is thrown, hence not all nodes get removed.
            for node_id in node_ids_in_groupings:
                try:
                    grouping.DelNode(node_id)
                except Exception:
                    pass
            for NI in grouping.Nodes():
                node_ids_in_groupings.append(NI.GetId())
            if not grouping.Empty():
                self.groupings[label] = grouping

        # We must place nodes with degree zero in their own grouping
        in_deg_v = self.network.GetNodeInDegV()
        out_deg_v = self.network.GetNodeOutDegV()
        zero_deg_nodes = snap.TIntV()
        for node_id_in_deg in in_deg_v:
            if node_id_in_deg.GetVal2() == 0:
                zero_deg_nodes.Add(node_id_in_deg.GetVal1())
        for node_id_in_deg in out_deg_v:
            if node_id_in_deg.GetVal2() != 0:
                zero_deg_nodes.DelIfIn(node_id_in_deg.GetVal1())
        if not zero_deg_nodes.Empty():
            grouping = self.network.ConvertSubGraph(snap.TNEANet, zero_deg_nodes)
            self.groupings[__no_label__] = grouping


# Evaluation

In [98]:
class SuperNode:
    super_node_id: int
    grouping_label: str
    sub_graph: snap.TNEANet
    
    def __init__(self, super_node_id, grouping_label, sub_graph):
        self.super_node_id = super_node_id
        self.grouping_label = grouping_label
        self.sub_graph = sub_graph


class Evaluation:
    def __init__(self, session):
        self.session = session
        self.super_edge_id_counter = itertools.count(start=1)
        self.super_node_id_counter = itertools.count(start=1)
        self.evaluation_graph = snap.TNEANet.New()

        # Node attributes
        self.evaluation_graph.AddFltAttrN(
            AttributeConstants.SN_NODE_WEIGHT.value)
        self.evaluation_graph.AddFltAttrN(
            AttributeConstants.SN_EDGE_WEIGHT.value)
        for label in self.session.labels_freq.keys():
            self.evaluation_graph.AddFltAttrN(
                AttributeConstants.SN_LABEL_PERCENTAGE.value + label)
            self.evaluation_graph.AddFltAttrN(
                AttributeConstants.SN_LABEL_REACH.value + label)
        self.evaluation_graph.AddIntAttrN(
            AttributeConstants.HYPER_NODE_ID.value)

        # Edge attributes
        self.evaluation_graph.AddStrAttrE(
            AttributeConstants.EDGE_LABEL.value)
        self.evaluation_graph.AddFltAttrE(
            AttributeConstants.SE_EDGE_WEIGHT.value)
        
        self.super_nodes = {}


    def add_super_node(self, inner_node_ids, grouping_label):
        super_node_id = next(self.super_node_id_counter)

        sub_graph = self.session.network.ConvertSubGraph(snap.TNEANet, 
            inner_node_ids)
        super_node = SuperNode(super_node_id, grouping_label, sub_graph)

        self.super_nodes[super_node_id] = super_node

        # Add an attribute to the nodes in the original graph indicating the 
        # the super node they are a part of
        for node_id in inner_node_ids:
            self.session.network.AddIntAttrDatN(
                node_id, super_node_id, 
                AttributeConstants.SUPER_NODE_ID.value)

        return super_node_id


    def evaluate(self):
        for label, grouping in self.session.groupings.items():
            # print("\t+++ Size of %s grouping: %d" % (label, grouping.GetNodes()))
            wccs = grouping.GetWccs()
            for wcc in wccs:
                # print("\t\t+++ Size of subgrouping: %d" % wcc.Len())
                inner_node_ids = snap.TIntV()
                for node_id in wcc:
                    inner_node_ids.Add(node_id)

                super_node_id = self.add_super_node(inner_node_ids, label)


    def build_evaluation_graph(self):
        labels = self.session.labels_freq.keys()
        network = self.session.network

        # Add all super nodes to evaluation graph
        for super_node_id in self.super_nodes.keys():
            self.evaluation_graph.AddNode(super_node_id)

        # Compute super node attributes and its super edges
        for super_node_id, super_node in self.super_nodes.items():
            labels_to_inner_edge_ids = {}
            labels_to_outer_edge_ids = {}
            super_node_connection_counter = {}
            for label in labels:
                labels_to_inner_edge_ids[label] = snap.TIntV()
                labels_to_outer_edge_ids[label] = snap.TIntV()
            
            for EI in super_node.sub_graph.Edges():
                src_node_id = EI.GetSrcNId()
                dst_node_id = EI.GetDstNId()
                edge_id = network.GetEI(src_node_id, dst_node_id).GetId()
                attr_values = snap.TStrV()
                network.AttrValueEI(edge_id, attr_values)
                label = attr_values[0]

                labels_to_inner_edge_ids[label].append(edge_id)
            
            # Add all the stats on the super node in the form of attributes
            # (1) Compression
            # Number of nodes inside super node
            node_weight = super_node.sub_graph.GetNodes()
            self.evaluation_graph.AddFltAttrDatN(super_node_id, node_weight, 
                AttributeConstants.SN_NODE_WEIGHT.value)

            # Number of edges inside super node
            edge_weight = super_node.sub_graph.GetEdges()
            self.evaluation_graph.AddFltAttrDatN(super_node_id, edge_weight, 
                AttributeConstants.SN_EDGE_WEIGHT.value)

            # (2) Inner-Connectivity
            # Store label frequency percentage and calculate reachability
            for label, edge_ids in labels_to_inner_edge_ids.items():
                try:
                    percentage = edge_ids.Len() / edge_weight
                except ZeroDivisionError:
                    percentage = 0
                self.evaluation_graph.AddFltAttrDatN(super_node_id, percentage, 
                    AttributeConstants.SN_LABEL_PERCENTAGE.value + label)
                
                reach = 0
                if edge_ids.Len() > 0:
                    label_sub_graph_sn = network.ConvertESubGraph(snap.TNEANet, edge_ids)
                    # CHECK used to be GetESubGraph(edge_ids)
                    for NI in label_sub_graph_sn.Nodes():
                        node_id = NI.GetId()
                        bfs_tree = label_sub_graph_sn.GetBfsTree(
                            node_id, True, False)
                        reach = reach + bfs_tree.GetEdges()
                self.evaluation_graph.AddFltAttrDatN(super_node_id, reach, 
                    AttributeConstants.SN_LABEL_REACH.value + label)
            
            # (3) Outer-Connectivity
            # computeConcatenationProperties what this function does still 
            # needs implementation
            
            for NI in super_node.sub_graph.Nodes():
                src_node_id = NI.GetId()
                src_super_node_id = network.GetIntAttrDatN(src_node_id,
                    AttributeConstants.SUPER_NODE_ID.value)
                # CHECK - NI.GetOutEdges exists apparently, it returns the edge id
                # get src out degree then for i in range(out_degree) NI.GetOutEId(i)
                # BUG since this returns the IDs, if there are multiple edges between
                # two nodes, this will only return the id once
                _, node_ids_at_one_hop = network.GetNodesAtHop(src_node_id, 1, True)
                for dst_node_id in node_ids_at_one_hop:
                    dst_super_node_id = network.GetIntAttrDatN(dst_node_id,
                        AttributeConstants.SUPER_NODE_ID.value)
                    if src_super_node_id != dst_super_node_id:
                        edge_id = network.GetEI(src_node_id, dst_node_id).GetId()
                        attr_values = snap.TStrV()
                        network.AttrValueEI(edge_id, attr_values)
                        label = attr_values[0]
                        super_node_connection_counter[(dst_super_node_id, label)] = (
                            super_node_connection_counter.setdefault(
                                (dst_super_node_id, label), 0) + 1)

            # Add super_edges and their attributes
            for (dst_super_node_id, label), edge_weight in super_node_connection_counter.items():
                # Add super edge to evaluation graph
                super_edge_id = self.evaluation_graph.AddEdge(super_node_id, dst_super_node_id)
                
                # Add label of the super edge
                self.evaluation_graph.AddStrAttrDatE(super_edge_id, 
                    label, AttributeConstants.EDGE_LABEL.value)

                # Number of edges inside super edge
                self.evaluation_graph.AddFltAttrDatE(super_edge_id, 
                    edge_weight, AttributeConstants.SE_EDGE_WEIGHT.value)


# Merge

In [99]:
# CHECK same as super node
class HyperNode:
    hyper_node_id: int
    reach_label: str
    sub_graph: snap.TNEANet

    def __init__(self, hyper_node_id, reach_label, sub_graph):
        self.hyper_node_id = hyper_node_id
        self.reach_label = reach_label
        self.sub_graph = sub_graph


class SuperNodeMergeInfo():
    def __init__(self, node_id, in_degree, out_degree, in_edge_labels, out_edge_labels, reach_label, reach_label_value):
        self.node_id = node_id
        self.in_degree = in_degree
        self.out_degree = out_degree
        self.in_edge_labels = in_edge_labels
        self.out_edge_labels = out_edge_labels
        self.reach_label = reach_label
        self.reach_label_value = reach_label_value


class SuperEdgeMergeInfo():
    def __init__(self, src_hyper_node_id, src_super_node_id, src_super_node_weight, dst_hyper_node_id, dst_super_node_id, dst_super_node_weight, super_edge_weight, super_edge_label):
        self.src_hyper_node_id = src_hyper_node_id
        self.src_super_node_id = src_super_node_id # not needed
        self.src_super_node_weight = src_super_node_weight # never used by the author
        self.dst_hyper_node_id = dst_hyper_node_id
        self.dst_super_node_id = dst_super_node_id # not needed
        self.dst_super_node_weight = dst_super_node_weight # used with edge_ratio
        # self.super_edge_ratio = super_edge_ratio # CHECK edge_ratio is used by the author for participation; I don't think it's needed
        self.super_edge_weight = super_edge_weight
        self.super_edge_label = super_edge_label


# We have two merging strategies
# Abstract merge class
# Two classes for each strategy
# Target merge - sink nodes have in_degree = 0
# Source merge - sink nodes have out_degree = 0
# We are focusing on target merge for now
class Merge:
    def __init__(self, session, evaluation):
        self.session = session
        self.evaluation = evaluation
        self.merge_graph = snap.TNEANet.New()
        self.hyper_edge_id_counter = itertools.count(start=1)
        self.hyper_node_id_counter = itertools.count(start=1)
        self.hyper_nodes = {}
        
        # Node attributes
        self.merge_graph.AddFltAttrN(
            AttributeConstants.HN_AVG_NODE_WEIGHT.value)
        
        # Edge attributes
        self.merge_graph.AddStrAttrE(
            AttributeConstants.EDGE_LABEL.value)
        # self.merge_graph.AddFltAttrE(
        #     EdgeAttributeConstants.EDGE_WEIGHT.value)
        
        self.super_nodes_merged_cnt = 0
        self.super_edges_merged_cnt = 0
    

    # CHECK same as super node
    def add_hyper_node(self, reach_label, super_node_ids):
        hyper_node_id = next(self.hyper_node_id_counter)
        self.super_nodes_merged_cnt += super_node_ids.Len()

        sub_graph = self.evaluation.evaluation_graph.ConvertSubGraph(
            snap.TNEANet, super_node_ids)
        hyper_node = HyperNode(hyper_node_id, reach_label, sub_graph)

        self.hyper_nodes[hyper_node_id] = hyper_node

        # Add an attribute to the nodes in the evaluation graph indicating the 
        # the hyper node they are a part of
        # CHECK maybe add this info to the OG graph too
        for super_node_id in super_node_ids:
            self.evaluation.evaluation_graph.AddIntAttrDatN(
                super_node_id, hyper_node_id, 
                AttributeConstants.HYPER_NODE_ID.value)
        
        return hyper_node_id
        

    def get_highest_reachability(self, node_id):
        reach_label = ""
        reach_label_value = 0
        attr_names = snap.TStrV()
        self.evaluation.evaluation_graph.FltAttrNameNI(node_id, attr_names)
        for attr_name in attr_names:
            if not attr_name.startswith(AttributeConstants.SN_LABEL_REACH.value):
                continue
            attr_value = self.evaluation.evaluation_graph.GetFltAttrDatN(
                node_id, attr_name)
            if attr_value > reach_label_value:
                # CHECK: create separator global and function to get label
                reach_label = attr_name.split("_")[-1]
                reach_label_value = attr_value
        
        return reach_label, reach_label_value


    def get_super_node_merge_info(self):
        super_nodes_merge_info = []

        # returns the in-degree for every node (node_id, degree)
        # node_id_in_degree = self.evaluation.evaluation_graph.GetNodeInDegV()
        for NI in self.evaluation.evaluation_graph.Nodes():
            node_id = NI.GetId()
            in_degree = NI.GetInDeg()
            out_degree = NI.GetOutDeg()
            in_edge_labels = set()
            out_edge_labels = set()

            # CHECK refactor this
            # get in edge labels
            for i in range(in_degree):
                edge_id = NI.GetInEId(i)
                label = self.evaluation.evaluation_graph.GetStrAttrDatE(
                    edge_id, AttributeConstants.EDGE_LABEL.value)
                in_edge_labels.add(label)

            # get out edge labels
            for i in range(out_degree):
                edge_id = NI.GetOutEId(i)
                label = self.evaluation.evaluation_graph.GetStrAttrDatE(
                    edge_id, AttributeConstants.EDGE_LABEL.value)
                out_edge_labels.add(label)
            
            reach_label, reach_label_value = self.get_highest_reachability(
                node_id)
            
            super_nodes_merge_info.append(SuperNodeMergeInfo(
                node_id, in_degree, out_degree, 
                in_edge_labels, out_edge_labels, 
                reach_label, reach_label_value))
        
        return super_nodes_merge_info
    

    # CHECK: Specify the strategy we want
    def make_hyper_nodes(self, super_nodes_merge_info, verbose=False):
        # we first need to sort the merge info according to the strategy used
        super_nodes_merge_info.sort(key=lambda merge_info: (
            merge_info.reach_label, merge_info.in_edge_labels))
        
        # then we group the objects according to the strategy used
        # CHECK what if the grouped super nodes have in/out edges into eachother?
        # from the example: v3 -l4> v2; v4 -l4> v3; v5 -l4> v4...
        super_node_groups = itertools.groupby(super_nodes_merge_info, 
            lambda merge_info: (merge_info.reach_label, merge_info.in_edge_labels))

        # CHECK reach_label_value is not used for anything at the moment, 
        # but the OG project had an option to group by that
        for key, group in super_node_groups:
            reach_label = key[0]
            grouped_super_nodes = snap.TIntV()

            for merge_info in group:
                node_id = merge_info.node_id
                grouped_super_nodes.append(node_id)
                
            hyper_node_id = self.add_hyper_node(reach_label, grouped_super_nodes)
            
            # CHECK we can probably calculate attributes now
        
        if verbose:
            for hn_id, hn in self.hyper_nodes.items():
                print(f"HN_id {hn_id}:")
                print(f"\tReach label: {hn.reach_label};")
                print(f"\tSubgraph nodes: {hn.sub_graph.GetNodes()};")
                for NI in hn.sub_graph.Nodes():
                    print(f"\t\tNode id: {NI.GetId()}.")
                print(f"\tSubgraph edges: {hn.sub_graph.GetEdges()}.")
                for EI in hn.sub_graph.Edges():
                    print(f"\t\tEdge id: {EI.GetId()}; ({EI.GetSrcNId()}) -> ({EI.GetDstNId()})")


    def build_hyper_edges(self, hyper_nodes_super_edges):
        # CHECK Given that we need to group stuff anyway, why not just make 
        # a list and also group by src_hyper_node_id
        for src_hyper_node_id, super_edges_merge_info in hyper_nodes_super_edges.items():
            # we first need to sort the list of s-edge merge info
            super_edges_merge_info.sort(key=lambda merge_info: (
                merge_info.dst_hyper_node_id, 
                merge_info.super_edge_label))
            
            # with the sorting done we can now group
            super_edge_groups = itertools.groupby(super_edges_merge_info, 
                lambda merge_info: (
                    merge_info.dst_hyper_node_id, 
                    merge_info.super_edge_label))
            
            # at this point group is a list of s-edge info with the same:
            # src_hyper_node, dst_hyper_node, edge_label
            for key, group in super_edge_groups:
                dst_hyper_node_id = key[0]
                label = key[1]

                sum_super_edge_weight = 0
                for merge_info in group:
                    sum_super_edge_weight += merge_info.super_edge_weight
                    
                    self.super_edges_merged_cnt += 1
                
                hyper_edge_id = next(self.hyper_edge_id_counter)

                # Add hyper edge to the merge graph
                hyper_edge_id = self.merge_graph.AddEdge(
                    src_hyper_node_id, dst_hyper_node_id)

                # Add label of the super edge
                self.merge_graph.AddStrAttrDatE(
                    hyper_edge_id, 
                    label,
                    AttributeConstants.EDGE_LABEL.value)

                # Number of edges inside super edge
                self.merge_graph.AddFltAttrDatE(
                    hyper_edge_id, 
                    sum_super_edge_weight, 
                    AttributeConstants.HE_EDGE_WEIGHT.value)


    def build_merge_graph(self):
        labels = self.session.labels_freq.keys()
        evaluation_graph = self.evaluation.evaluation_graph
        hyper_nodes_super_edges = {} # where we will info on the super edges that need merging

        # Add all hyper nodes to the merge graph
        for hyper_node_id in self.hyper_nodes.keys():
            self.merge_graph.AddNode(hyper_node_id)
        
        # Compute hyper node attributes and its hyper edges
        for hyper_node_id, hyper_node in self.hyper_nodes.items():
            node_attributes = {}

            # Subgraph does not have the node attributes because fuck me right?
            for NI in hyper_node.sub_graph.Nodes():
                # Sum the node weight among all super nodes
                attr_name = AttributeConstants.SN_NODE_WEIGHT.value
                sn_node_weight = evaluation_graph.GetFltAttrDatN(NI, attr_name)
                node_attributes[attr_name] = (
                    node_attributes.setdefault(attr_name, 0) + sn_node_weight)
                
                # Sum the edge weight among all super nodes
                attr_name = AttributeConstants.SN_EDGE_WEIGHT.value
                sn_edge_weight = evaluation_graph.GetFltAttrDatN(NI, attr_name)
                node_attributes[attr_name] = (
                    node_attributes.setdefault(attr_name, 0) + sn_edge_weight)

                # Now we handle the attributes that relate to each label
                for label in labels:
                    # Sum the label reach for each label in the super nodes
                    attr_name = AttributeConstants.SN_LABEL_REACH.value + label
                    sn_label_reach = evaluation_graph.GetFltAttrDatN(NI, attr_name)
                    node_attributes[attr_name] = (
                        node_attributes.setdefault(attr_name, 0) + sn_label_reach)
                    
                    # For the percentage of inner edge labels we need to multiply by the total edges
                    # CHECK: why store the percentage and not just the total?
                    attr_name = AttributeConstants.SN_LABEL_PERCENTAGE.value + label
                    sn_label_percentage = evaluation_graph.GetFltAttrDatN(NI, attr_name)
                    node_attributes[attr_name] = (
                        node_attributes.setdefault(attr_name, 0) 
                        + (sn_edge_weight * sn_label_percentage))

                # Hyper edge stuff CHECK strategy
                src_super_node_id = NI.GetId()
                src_hyper_node_id = hyper_node_id
                _, super_node_ids_at_one_hop = evaluation_graph.GetNodesAtHop(
                    src_super_node_id, 1, True)
                src_super_node_weight = evaluation_graph.GetFltAttrDatN(
                    src_super_node_id, 
                    AttributeConstants.SN_NODE_WEIGHT.value)

                # CHECK - NI.GetOutEdges exists apparently, it returns the edge id
                # get src out degree then for i in range(out_degree) NI.GetOutEId(
                for dst_super_node_id in super_node_ids_at_one_hop:
                    super_edge_id = evaluation_graph.GetEI(
                        src_super_node_id, dst_super_node_id).GetId()
                    dst_hyper_node_id = evaluation_graph.GetIntAttrDatN(
                        dst_super_node_id, 
                        AttributeConstants.HYPER_NODE_ID.value)
                    dst_super_node_weight = evaluation_graph.GetFltAttrDatN(
                        dst_super_node_id,
                        AttributeConstants.SN_NODE_WEIGHT.value)

                    # CHECK super_edge_ratio
                    # used by the author to calculate participation but I don't think it's needed

                    super_edge_weight = evaluation_graph.GetFltAttrDatE(
                        super_edge_id, AttributeConstants.SE_EDGE_WEIGHT.value)
                    super_edge_label = evaluation_graph.GetStrAttrDatE(
                        super_edge_id, AttributeConstants.EDGE_LABEL.value)

                    # Store super edge merge info
                    super_edge_merge_info = SuperEdgeMergeInfo(
                        src_hyper_node_id, src_super_node_id, 
                        src_super_node_weight, dst_hyper_node_id, 
                        dst_super_node_id, dst_super_node_weight, 
                        super_edge_weight, super_edge_label)
                    
                    hyper_nodes_super_edges.setdefault(
                        src_hyper_node_id, []).append(super_edge_merge_info)
            
            # Now that we have sum of the (edge label% * edge_weight) of every 
            # super node, we divide it by the total_edge_weight (i.e. the edge 
            # weight of the hyper node)
            attr_name = AttributeConstants.SN_EDGE_WEIGHT.value
            total_sn_edge_weight = node_attributes[attr_name]
            for label in labels:
                # CHECK: why store the percentage and not just the total?
                attr_name = AttributeConstants.SN_LABEL_PERCENTAGE.value + label
                try:
                    percentage = node_attributes[attr_name] / total_sn_edge_weight
                except ZeroDivisionError:
                    percentage = 0
                node_attributes[attr_name] = percentage

            # Number of super nodes inside hyper node
            attr_name = AttributeConstants.HN_NODE_WEIGHT.value
            hn_node_weight = hyper_node.sub_graph.GetNodes()
            node_attributes[attr_name] = hn_node_weight

            # Average number of nodes inside each super node of the hyper node
            attr_name = AttributeConstants.SN_NODE_WEIGHT.value
            total_node_weight = node_attributes[attr_name]
            attr_name = AttributeConstants.HN_AVG_NODE_WEIGHT.value
            avg_node_weight = total_node_weight / hn_node_weight
            node_attributes[attr_name] = avg_node_weight
            
            # CHECK Still need to calculate frontier attributes

            # Add all the properties to the hyper node
            for attr_name, attr_value in node_attributes.items():
                self.merge_graph.AddFltAttrDatN(
                    hyper_node_id, attr_value, attr_name)

        # Now we need to create the super edges
        self.build_hyper_edges(hyper_nodes_super_edges)


    def merge(self):
        # getting super node info in order to merge them
        # we get all the info no matter the strategy used
        super_nodes_merge_info = self.get_super_node_merge_info()
        
        # Adding hyper nodes to the respective class dict
        # Here we need to specify the strategy we want
        self.make_hyper_nodes(super_nodes_merge_info, verbose=False)

        self.build_merge_graph()


# Main

In [100]:
print("--> Loading dataset...")
# network = make_pg_paper_network()
# network = make_residence_hall_network()
network = make_author_repo_network(GMarkUseCase.uniprot)
print("--> Dataset loaded.")
print(f"\t+++ Number of Nodes: {network.GetNodes()}")
print(f"\t+++ Number of Edges: {network.GetEdges()}")
print("--> Creating session...")
session = Session(network)
print("--> Computing groupings...")
session.compute_groupings()
print("--> Preparing for evaluation...")
evaluation = Evaluation(session)
print("--> Evaluating...")
evaluation.evaluate()
print("--> Building evaluation graph...")
evaluation.build_evaluation_graph()
print("--> Evaluation completed.")
print(f"\t+++ Number of super nodes: {evaluation.evaluation_graph.GetNodes()}")
print(f"\t+++ Number of super edges: {evaluation.evaluation_graph.GetEdges()}")
print("--> Preparing for merging...")
merge = Merge(session, evaluation)
print("--> Merging...")
merge.merge()
print("--> Merging completed.")
print(f"\t+++ Number of hyper nodes: {merge.merge_graph.GetNodes()}")
print(f"\t+++ Number of hyper edges: {merge.merge_graph.GetEdges()}")

--> Loading dataset...
--> Dataset loaded.
	+++ Number of Nodes: 342
	+++ Number of Edges: 363
--> Creating session...
--> Computing groupings...
--> Preparing for evaluation...
--> Evaluating...
--> Building evaluation graph...
--> Evaluation completed.
	+++ Number of super nodes: 91
	+++ Number of super edges: 92
--> Preparing for merging...
--> Merging...
--> Merging completed.
	+++ Number of hyper nodes: 8
	+++ Number of hyper edges: 11


In [101]:
# print_all_node_attributes(merge.merge_graph)
# print_all_edge_attributes(merge.merge_graph)
check_merge_graph(network, evaluation.evaluation_graph, merge.merge_graph)

363 -> 103.0 + 257.0 -> 103.0 + ???


In [106]:
print(network.GetIntAttrDatN(4, AttributeConstants.SUPER_NODE_ID.value))

9


In [29]:
Graph = merge.merge_graph
labels = {}
for NI in Graph.Nodes():
    labels[NI.GetId()] = str(NI.GetId())
Graph.DrawGViz(snap.gvlDot, "output.png", " ", labels)

In [ ]:
labels_edge_ids = {}
for EI in pg_paper.Edges():
    edge_id = EI.GetId()
    attr_values = snap.TStrV()
    pg_paper.AttrValueEI(edge_id, attr_values)
    label = attr_values[0]
    labels_edge_ids.setdefault(label, snap.TIntV()).append(edge_id)

sub_graph = pg_paper.GetESubGraph(labels_edge_ids['l0'])

# shortestPath, NIdToDistH = sub_graph.GetShortPathAll(2, IsDir=True)
# for item in NIdToDistH:
#     print(item, NIdToDistH[item])
# print(shortestPath)

reach = 0
for NI in sub_graph.Nodes():
    node_id = NI.GetId()
    bfs_tree = sub_graph.GetBfsTree(node_id, True, False)
    reach = reach + bfs_tree.GetEdges()
    print(node_id, reach)

# Archive

In [ ]:
# def merge_sink_nodes(self):
#         super_nodes_highest_reach = {}

#         # returns the in-degree for every node (node_id, degree)
#         node_id_in_degree = self.evaluation.evaluation_graph.GetNodeInDegV()
#         for (node_id, in_degree) in node_id_in_degree:
#             if in_degree != 0:
#                 continue

#             reach_label = ""
#             reach_label_value = -1.0
#             attr_names = snap.TStrV()
#             self.evaluation.evaluation_graph.FltAttrNameNI(node_id, attr_names)
#             for attr_name in attr_names:
#                 if not attr_name.startswith(LabelAttributeConstants.REACH.value):
#                     continue
#                 attr_value = self.evaluation.evaluation_graph.FltAttrValueNI(
#                     node_id, attr_name)
#                 if attr_value > reach_label_value:
#                     # CHECK: create separator global and function to get label
#                     reach_label = attr_name.split("_")[-1]
#                     reach_label_value = attr_value
                    
#             if (reach_label_value > 0):
#                 super_nodes_highest_reach[node_id] = (
#                     reach_label, reach_label_value)

#         super_nodes_by_reach_label = {}
#         # CHECK reach_label_value is not used for anything at the moment, 
#         # but the OG project had an option to group by that
#         for super_node_id, (reach_label, _) in super_nodes_highest_reach.item():
#             super_nodes_by_reach_label.setdefault(
#                 reach_label, snap.TIntV()).append(super_node_id)

#         for reach_label, super_node_ids in super_nodes_by_reach_label.items():
#             hyper_node_id = self.add_hyper_node(reach_label, super_node_ids)
#             self.super_nodes_merged_cnt += super_node_ids.Len()

#              # PERHAPS Add an attribute to the super nodes and the nodes in the
#              # original graph indicating the the hyper node they are a part of